# Transmission SAXS Analysis for Operando Membrane Fouling Experiments
For single (i.e., non-tiled) images collected at ALS beamline 7.3.3

In [ ]:
import warnings; warnings.filterwarnings('ignore')

In [ ]:
import os
import re
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline 
import pyFAI
import fabio
plt.style.use('default')
from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from matplotlib.colorbar import Colorbar as colorbar
from matplotlib.colors import LogNorm
from scipy.signal import find_peaks
from scipy.integrate import simps
from tqdm import tqdm


In [ ]:
flag_save = True

In [ ]:
# using  sorted, sample10 comes before sample1, so we clean this up now - taken from: https://stackoverflow.com/questions/5967500/how-to-correctly-sort-a-string-with-a-number-inside
def text_to_int(text): return int(text) if text.isdigit() else text
def natural_keys(text): return [ text_to_int(c) for c in re.split(r'(\d+)', text ) ]

In [ ]:
# locate files

master_path = '/global/cfs/cdirs/als/mwet/data/Landsman Temp/operando instrument paper/'
sample_folder = 'exp04a'

data_path = os.path.join(master_path, 'saxs_data', sample_folder)
save_path = os.path.join(master_path, 'saxs_analysis', sample_folder)
if flag_save: os.makedirs(save_path, exist_ok=True)


In [ ]:
# load data

files = sorted([os.path.join(data_path, f) for f in os.listdir(os.path.join(data_path)) if 'restart' not in f and 'beamstop_test' not in f and 'autoexpose_test' not in f and 'DS_Store' not in f and '' in f])
files.sort(key = natural_keys)

imagefiles = sorted([os.path.join(data_path, f) for f in files if '.edf' in f and 'fouling' in f], key=lambda x:  files.index(x))
txtfiles = sorted([os.path.join(data_path, f) for f in files if '.txt' in f and 'fouling' in f], key=lambda x:  files.index(x))
sample_list = pd.Series([f[ : f.find('_2m.edf')] for f in imagefiles]).unique()
sample_list_fname = pd.Series([f[f.find(data_path)+len(data_path):] for f in sample_list]).unique()

print(pd.Series(sample_list_fname))

In [ ]:
# plot 2D images

for samp in tqdm((sample_list[:])):
    fname = sorted(f for f in imagefiles if (samp in f))[0]    
    file = os.path.join(data_path,fname)
    image = fabio.open(file)
    array = image.data
    array[array < 1] = 1

    fig, ax = plt.subplots()
    im = ax.imshow(array, norm=LogNorm(vmin=np.percentile(array, 20), vmax=np.percentile(array, 99.9)))

    sample_title = os.path.basename(fname)
    ax.set_title(sample_title)
    ax.axis('off')
        
    ax_divider = make_axes_locatable(ax)
    cax = ax_divider.append_axes('bottom', size='3%', pad='2%')
    plt.colorbar(im, cax=cax, orientation='horizontal')
    cax.set_xlabel('Scattering intensity (arbitrary units)', size=8)
    cax.xaxis.set_label_position('bottom')
    cax.xaxis.tick_bottom()
    cax.xaxis.set_tick_params(labelsize=8)   
    fig.tight_layout()
    
    if flag_save:
        save_path_2dimages = os.path.join(save_path, 'saxs_2dimages')
        os.makedirs(save_path_2dimages, exist_ok=True)
        save_fname = os.path.join(save_path_2dimages, os.path.basename(samp) + '_2dimage.png')
        fig.savefig(save_fname, bbox_inches='tight', dpi=300)    
    

In [ ]:
# azimuthal integrator (this is the most computationally expensive component, so we load it once and refer to it within loops later)
# poni file was created using pyFAI calibration GUI (https://pyfai.readthedocs.io/en/master/usage/cookbook/calib-gui/index.html)

poni = os.path.join(master_path, 'saxs_data', 'saxs_poni_mrl.poni')
ai = pyFAI.load(poni)
ai.maskfile = os.path.join(master_path, 'saxs_data', 'saxs_mask_mrl.edf')

ai

In [ ]:
# batch process(+save) reduced profiles --> normalize based on diode value, radially integrate, plot 1D profiles    

# input q values expected for nanoparticles (to draw vertical lines) 
specific_q_values = [0.01185, 0.01843, 0.02496, 0.03098, 0.03805]

colormap = plt.get_cmap('Blues')
colors = colormap(np.linspace(0.01, 1, len(sample_list)))
z = np.zeros((1, 2000))
azimuth_range = [-180, 180]

fig, ax = plt.subplots(figsize=(6, 6))

timestep_list = []
tsi_list = []

specific_q_value = 0.01
intensities_for_specific_q = []

for idx, samp in tqdm(enumerate(sample_list[30:-2])):
    
    ctrl_fname = imagefiles[29]
    ctrl_txtfile = txtfiles[29]
    ctrl_I1_value = int(np.loadtxt(os.path.join(data_path, ctrl_txtfile), skiprows=1, max_rows=1))
    ctrl_diode_value = int(np.loadtxt(os.path.join(data_path, ctrl_txtfile), skiprows=2, max_rows=1))
    ctrl_file = os.path.join(data_path, ctrl_fname)
    ctrl_image = fabio.open(ctrl_file)
    ctrl_array = ctrl_image.data
    ctrl_array[ctrl_array < 1] = 1
    ctrl_reduced = ai.integrate1d(ctrl_array, npt=2000, azimuth_range=azimuth_range)    
    q_ctrl = ctrl_reduced[0] / 10
    i_ctrl = ctrl_reduced[1] / ctrl_diode_value

    txtfile = sorted(t for t in txtfiles if (samp in t))[0]
    I1_value = int(np.loadtxt(txtfile, skiprows=1, max_rows=1))
    diode_value = int(np.loadtxt(txtfile, skiprows=2, max_rows=1))
    fname = sorted(f for f in files if (samp in f))[0]
    sample_file = os.path.join(data_path, fname)
    sample_image = fabio.open(sample_file)
    sample_array = sample_image.data
    sample_array[sample_array < 1] = 1
    sample_reduced = ai.integrate1d(sample_array, npt=2000, azimuth_range=azimuth_range)    
    q_sample = sample_reduced[0] / 10
    i_sample = sample_reduced[1] / diode_value    
    i_sample_norm = i_sample - i_ctrl*0.9
    
    # input q boundaries
    q_min = 0.009076
    q_max = 0.01437
    
    q_indices = (q_sample >= q_min) & (q_sample <= q_max)
    tsi_q_sample = q_sample[q_indices]
    tsi_i_sample_norm = i_sample[q_indices]
    
    tsi_valid_indices = ~np.isnan(tsi_i_sample_norm)
    tsi = np.trapz(tsi_i_sample_norm[tsi_valid_indices], tsi_q_sample[tsi_valid_indices])
    tsi_list.append(tsi)
    timestep_list.append(idx)
    
    i_sample_krakty = i_sample_norm * q_sample**2
    ax.plot(q_sample, i_sample_krakty, '-', linewidth=1, markerfacecolor='none', color=colors[idx])
    ax.spines['top'].set_linewidth(1.5)
    ax.spines['right'].set_linewidth(1.5)
    ax.spines['bottom'].set_linewidth(1.5)
    ax.spines['left'].set_linewidth(1.5)
    ax.set_xlabel(r'Scattering vector, $q \ (\mathrm{\AA^{-1}})$', fontsize=20)
    ax.set_ylabel(r'Scattering intensity * $q^2$ (arb)', fontsize=20)
    ax.tick_params(axis='both', which='major', labelsize=18, width=2, length=6, direction='in', labelcolor='black')
    ax.minorticks_on()
    ax.tick_params(axis='x', which='minor', length=4, width=1.5, direction='in', labelsize=0)  # Hide minor tick labels
    plt.xticks(fontsize=20)
    plt.yticks(fontsize=20)
    ax.set_xscale('log')
    ax.set_xlim([5e-3, 2.7e-2])
    ax.set_ylim([0, 0.00000055])
    plt.grid(False)
    
    
    
    
    
    
    
    z = np.vstack((z, i_sample_krakty))
    
    # Find the index of the closest q_val to the specific_x_value
    closest_index = np.argmin(np.abs(q_sample - specific_q_value))

    # Get the corresponding i_sample_norm value
    closest_int_value = i_sample_norm[closest_index]
    intensities_for_specific_q.append(closest_int_value)

# Add vertical lines at the specified x values
for x_val in specific_q_values:
    ax.axvline(x=x_val, color='red', linestyle='--', linewidth=1)

plt.legend()
plt.show()


In [ ]:
# lets take a look at the transmission signal on the diode as a function of time

fig, ax = plt.subplots(figsize=(6, 4))
ax.set_title(f'Transmission flux through flow cell\n{sample_folder}')
ax.plot(transmission_list, 'o-')
ax.set_xlabel('sample index')
ax.set_ylabel('photodiode flux / I1 flux')
ax.grid(which='major', color='lightgrey', linewidth=0.6); ax.grid(which='minor', color='lightgrey', linewidth=0.3)

if flag_save:
    save_plotname = os.path.join(save_path, sample_folder + '_transmissionflux.png')
    fig.savefig(save_plotname, bbox_inches='tight', dpi=300)
        


In [ ]:
# now lets take a look at the total scattering intensity

fig, ax = plt.subplots(figsize=(6, 4))
ax.set_title(f'Transmission flux through flow cell\n{sample_folder}')
ax.plot(tsi_list, 'o-')
ax.set_xlabel('sample index')
ax.set_ylabel('photodiode flux / I1 flux')
ax.grid(which='major', color='lightgrey', linewidth=0.6); ax.grid(which='minor', color='lightgrey', linewidth=0.3)

if flag_save:
    save_plotname = os.path.join(save_path, sample_folder + '_transmissionflux.png')
    fig.savefig(save_plotname, bbox_inches='tight', dpi=300)
        
